# 2023 AI/ML UnicornGym - Track 1

@jesamkim | 2023-Feb-08

# 8. 엔드포인트 배포 스텝 파이프라인

### 프로세싱 스텝 
- 엔드포인트 배포는 "deploy_model.py" 의 스크립트를 실행하여 배포를 합니다. 그래서 여기서는 "프로세싱 스텝" 을 사용합니다.
- 프로세싱 단계의 개발자 가이드 
    - [프로세싱 스텝](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)



### setup environment

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [2]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
%store -r 

%store

Stored variables and their in-db values:
bucket                           -> 'sagemaker-us-east-1-376278017302'
endpoint_name                    -> 'sagemaker-xgboost-2023-02-09-05-24-37-494'
hyperparameters                  -> {'scale_pos_weight': '19', 'max_depth': '2', 'eta'
image_uri                        -> '683313688378.dkr.ecr.us-east-1.amazonaws.com/sage
input_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
project_prefix                   -> 'titanic-ml'
s3_path                          -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
sagemaker_model                  -> 'pipelines-4jksbfxkgkny-survived-basic-creat-km39j
test_data_uri                    -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_df                          ->     PassengerId  Pclass  Sex   Age  SibSp  Parch  
train_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
train_model_artifact             -> 's3://sagemaker-us-east-1-3

---
## 8-1. 모델 빌딩 파이프라인의 스텝(Step) 생성

### 모델 빌딩 파이프라인 변수 생성

In [3]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)
endpoint_instance_type = ParameterString(
    name="EndpointInstanceType",
    default_value="ml.m5.xlarge"
)

### 배포에 사용할 스크립트 코드 S3 업로딩

In [4]:
from datetime import datetime
suffix = datetime.now().microsecond
print("suffix: ", suffix)


local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: \n", s3_deploy_code_uri)

pipeline_endpoint_name = 'pipeline-endpoint-'  + str(suffix)

suffix:  974922
s3_deploy_code_uri: 
 s3://sagemaker-us-east-1-376278017302/titanic-ml/code/deploy_model.py


### 배포에 사용할 프로세서 정의

In [5]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type= processing_instance_type, 
    instance_count= processing_instance_count,
    base_job_name='survived-scratch-deploy-model',
    sagemaker_session=sagemaker_session)



### 모델 엔드포인트 생성 스텝 생성

In [6]:
deploy_step = ProcessingStep(
    name='Survived-Basic-Endpoint',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", sagemaker_model, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

---
## 8-2. 파라미터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행

### 파이프라인 정의
[중요] `pipeline_endpoint_name` 에 '_' 언더바를 넣으면 에러가 납니다. '-' 대시는 가능합니다.

In [7]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        endpoint_instance_type
    ],
    steps=[deploy_step],
)

### 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [8]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'EndpointInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Survived-Basic-Endpoint',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerArguments': ['--model_name',
      'pipelines-4jksbfxkgkny-survived-basic-creat-km39jzydyp',
      '--regio

### 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [9]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [10]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml/execution/9dbribma4nq1',
 'PipelineExecutionDisplayName': 'execution-1675921169526',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 2, 9, 5, 39, 29, 348000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 2, 9, 5, 39, 29, 348000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'd496d5fe-50f3-44e7-a7eb-663f10450cb5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd496d5fe-50f3-44e7-a7eb-663f10450cb5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '385',
   'date': 'Thu, 09 Feb 2023 05:39:29 GMT'},
  'RetryAttempts': 0}}

### 파이프라인 실행 기다리기

In [11]:
execution.wait()

### 파이프라인 실행 단계 기록 보기

In [12]:
execution.list_steps()

[{'StepName': 'Survived-Basic-Endpoint',
  'StartTime': datetime.datetime(2023, 2, 9, 5, 39, 30, 600000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 9, 5, 45, 33, 387000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:376278017302:processing-job/pipelines-9dbribma4nq1-survived-basic-endpo-orygu75e8v'}}}]

---
## 8-3. 세이지 메이커 스튜디오에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
    - SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)



![endpoint-sm.png](img/endpoint-sm.png)

---
## 8-4. 엔드포인트 추론 테스트

### 추론에 사용할 Payload 생성 및 추론
- 테스트 데이터를 통해서 엔드포인트에 전달할 CSV 형태의 String을 생성합니다. (payload).
- payload 를 엔드포인트에 제공하면, 확률값을 0.072 을 리턴합니다. 
- 보통 확률값이 0.5 보다 작으면 0 (Non-Fruad), 0.5 보다 크면 1 (Fruad) 로 변환해서 예측값으로 사용합니다.

```
payload: 
 1,1,38.0,1,0,71.2833,1

Reponse: 
[['0.2789361774921417']]



```

### 테스트 데이터 준비

In [13]:
import boto3
sagemaker_boto_client = boto3.client('sagemaker')
from src.p_utils import get_predictor, get_payload, predict

from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.

In [14]:
test_preproc_path = f"{train_data_uri}"
test_df = pd.read_csv(test_preproc_path)
#test_df = test_df.drop('Unnamed: 0', axis=1)
test_df.head(1)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,2,1,1,1,38.0,1,0,1


### 엔드포인트에 입력이 될 데이터 (payload) 생성

In [15]:
# test_df 의 상위 1개 레코드를 사용
payload = get_payload(test_df, label_col = 'Survived', verbose=False)
print("payload: \n", payload)

payload: 
 2,1,1,38.0,1,0,1


### 추론 하여 예측 확률 얻기

In [16]:

predictor = get_predictor(pipeline_endpoint_name, sagemaker_session, csv_deserializer)

pred_prob = predict(predictor, payload)
print("prediction_prob: \n", pred_prob)


prediction_prob: 
 ['0.9955571293830872']


---
## 8-5. 엔드포인트 삭제
파이프라인 실행을 통하여 생성된 앤드포인트 및 관련 오브젝트 (세이지 메이커 모델, 엔드포인트 컨피그) 를 삭제 합니다.

In [17]:
from src.p_utils import delete_endpoint

delete_endpoint(sagemaker_boto_client, pipeline_endpoint_name, is_del_model = True )

--- Deleted model: pipelines-4jksbfxkgkny-survived-basic-creat-km39jzydyp
--- Deleted endpoint: pipeline-endpoint-974922
--- Deleted endpoint_config: pipelines-4jksbfxkgkny-survived-basic-creat-km39jzydyp-config
